In [ ]:
# Install memorizz and required dependencies
%pip install -qU memorizz

# Install Oracle database driver (required for Oracle provider)
%pip install -qU oracledb

# Install OpenAI SDK (for LLM and embeddings)
%pip install -qU openai

# Install requests (for tool examples like weather API)
%pip install -qU requests

# Install python-dotenv for .env file support (optional but recommended)
%pip install -qU python-dotenv

print("✅ All packages installed successfully!")


In [ ]:
! memorizz install-oracle

In [ ]:
    import os

    ORACLE_ADMIN_PASSWORD = "MyPassword123!"
    ORACLE_USER = "memorizz_user"
    ORACLE_PASSWORD = "SecurePass123!"
    ORACLE_DSN = "localhost:1521/FREEPDB1"

    os.environ["ORACLE_ADMIN_PASSWORD"] = "MyPassword123!"
    os.environ["ORACLE_USER"] = "memorizz_user"
    os.environ["ORACLE_PASSWORD"] = "SecurePass123!"
    os.environ["ORACLE_DSN"] = "localhost:1521/FREEPDB1"

In [ ]:
! memorizz setup-oracle

In [ ]:
import logging
import os

# Configure logging for Jupyter notebook
os.environ['MEMORIZZ_LOG_LEVEL'] = 'INFO'

# Set up proper logging configuration for notebooks
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True  # This overwrites any existing configuration
)

In [ ]:
import getpass

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OpenAI API key: ")

In [ ]:
set_env_securely("TAVILY_API_KEY", "Enter your Tavily API key: ")

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


In [ ]:
from memorizz.memory_provider.oracle import OracleProvider, OracleConfig


# Open source model option
HF_MODEL = "sentence-transformers/all-MiniLM-L6-v2" 
# Proprietary model option
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small" 

# HF_TOKEN = os.getenv("HF_TOKEN")  # optional if the model is public
huggingface_embedding_config = {
    "model": HF_MODEL,
    "api_key": os.getenv("HF_TOKEN"),
}

# Ensure you have a valid API key and it's set in your environment
openai_embedding_config = {
    "model": OPENAI_EMBEDDING_MODEL,
    "api_key": OPENAI_API_KEY,
}

# Creating a config object for the Oracle provider
oracle_config = OracleConfig(
    user=ORACLE_USER,
    password=ORACLE_PASSWORD,
    dsn=ORACLE_DSN,
    lazy_vector_indexes=False,
    embedding_provider="openai", # or "huggingface" if you're using the open source model
    embedding_config=openai_embedding_config, # or huggingface_embedding_config if you're using the open source model
)

# Creating the Oracle Memory provider using Oracle AI Database as the Memory Core
oracle_memory_provider = OracleProvider(oracle_config)

In [ ]:
from memorizz.memagent.builders import MemAgentBuilder

# Creating a config object for the Hugging Face LLM provider, use this if you're using the open source model path
huggingface_llm_config = {
    "provider": "huggingface",
    "model": "Qwen/Qwen2-1.5B-Instruct",  # fully open, CPU-friendly
    "tokenizer": "Qwen/Qwen2-1.5B-Instruct",  # optional (defaults to model id)
    "device": "cpu",          # use "cuda", "cuda:0", or "mps" if you have a GPU
    "max_new_tokens": 512,
    "temperature": 0.2,
    "top_p": 0.9,
}

# Creating a config object for the OpenAI LLM provider, use this if you're using the proprietary model path
openai_llm_config = {
    "model": "gpt-4o-mini",
    "api_key": OPENAI_API_KEY,
}



In [ ]:
from memorizz.internet_access import TavilyProvider

# Ensure you have a valid API key and it's set in your environment
tavily_internet_provider = TavilyProvider(
    api_key=TAVILY_API_KEY,
    config={
        "search_depth": "advanced",     # or "basic"
        "default_max_results": 8,       # cap per search() call
        "max_content_chars": 10000,     # trim extracted pages to protect context window
        "include_raw_results": False,   # True if you want Tavily’s raw JSON back
        "include_raw_page": False,
    },
)

```create_deep_research_agent(...)``` is a convenience helper that returns a MemAgentBuilder pre-set for Deep Research mode—hooked up with the provided (or default) internet provider, the standard multi-step research instruction, and the deep-research application mode—so you can chain on memory/config tweaks before calling .build().

In [ ]:
from memorizz.memagent.builders import create_deep_research_agent

def build_agent(instruction: str):
    return (
        # Agents are created with with Tavily as the internet provider
        create_deep_research_agent(instruction, internet_provider=tavily_internet_provider)
        # Agents are created with with Oracle as the memory provider
        .with_memory_provider(oracle_memory_provider)
        # Agents are created with with OpenAI as the LLM provider
        .with_llm_config(openai_llm_config)
        .build()
    )

In [ ]:
root_agent = build_agent(
    "You are the lead analyst. Break the Apple (AAPL) stock review into sub-questions "
    "covering financial performance, market share, and risk outlook over the last 3 years. "
    "Assign tasks, monitor progress, and coordinate synthesis."
)

delegate_financials = build_agent(
    "Financial specialist focusing on Apple’s revenue, profit, and cash flow trends "
    "from the last three fiscal years. Cite filings or press releases."
)

delegate_market = build_agent(
    "Market researcher gathering Apple’s market share, major competitors, and macro headwinds "
    "from the past three years."
)

delegate_risk = build_agent(
    "Risk analyst investigating regulatory, supply-chain, and innovation risks affecting Apple "
    "over the same period."
)

synthesis_agent = build_agent(
    "Synthesis expert who combines delegate reports into a single memo with sections for "
    "Financials, Market Landscape, and Risks. Include bullet-point insights plus references."
)

root_agent.save()
delegate_financials.save()
delegate_market.save()
delegate_risk.save()
synthesis_agent.save()


DeepResearchOrchestrator coordinates a MemAgent’s end‑to‑end research workflow—running the planner, delegates, and synthesis steps—so the agent can plan, browse, summarize, and report findings autonomously.


In [ ]:
from memorizz.memagent.orchestrators import DeepResearchOrchestrator

orchestrator = DeepResearchOrchestrator(
    root_agent=root_agent,
    delegates=[delegate_financials, delegate_market, delegate_risk],
    synthesis_agent=synthesis_agent,
)

In [ ]:
query = "Provide a three-year stock analysis for Apple Inc. (AAPL), highlighting financial trends, market dynamics, and key risks."

In [ ]:

report = orchestrator.execute(query)

In [ ]:
print(report)